In [1]:
import numpy as np

from DataIngestion.configurated_website_loaders import news_article_loader
from Preprocessing.text_extraction import SimpleBS4TextExtractor
from Preprocessing.text_spitting import RecursiveTextSplitter
from Preprocessing.image_loaders import RequestsImageLoader
from Preprocessing.image_describer import BLIPImageDescriber
from Embedding.text_embedding import SentenceTransformerTextEmbedding
from VectorStore.chroma_vector_store import ChoromaVectorStore
from Internals.adapters import ChromaTextEmbeddingAdapter
from LLM.rag_llm import OllamaRAGLLM

BASE_URL = "https://www.deeplearning.ai/the-batch"
NEW_URL = "https://www.deeplearning.ai/the-batch/issue-284/"

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\User\Desktop\MultimodalRAGSystem\LLM\rag_llm.py:58: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model: Ollama = pydantic.Field(default=Ollama(model='llama3.2'))


In [2]:
loaded_data = news_article_loader.load(NEW_URL)
loaded_data

ParsedData(url=https://www.deeplearning.ai/the-batch/issue-284/, parsed_tags=dict_keys(['Title', 'Author', 'Published_date', 'Content', 'Summary', 'Tags', 'Paragraph', 'Image']))

In [3]:
text_extractor = SimpleBS4TextExtractor()
image_loader = RequestsImageLoader()

text = text_extractor.extract_text_from_elements(loaded_data.get_all())
images = []
for img in loaded_data.image:
    loaded_img = image_loader.load(img['src'])
    if loaded_img is not None:
        images.append(loaded_img)

In [4]:
text

"issue 284\nNews\nDeepSeek Ups the Open Weights Ante\nU.S. Moves to Expand AI Export Restrictions\nAI Supercomputer on Your Desk\nCalibrating Contrast\n\n\nExplore Courses\nAI Newsletter\n\nCommunity\n\nResources\nCompany\n\n\n\n\n\n\n\n✨ New course! Enroll in Reinforcement Fine-Tuning LLMs with GRPO Explore Courses AI Newsletter The Batch Andrew's Letter Data Points ML Research Blog Community Forum Events Ambassadors Ambassador Spotlight Resources Company About Careers Contact Start Learning Weekly Issues Andrew's Letters Data Points ML Research Business Science Culture Hardware AI Careers About Subscribe The Batch Weekly Issues issue 284 Published Jan 15, 2025 Reading time 13 min read Published Jan 15, 2025 Reading time 13 min read Share Dear friends, Writing software, especially prototypes, is becoming cheaper. This will lead to increased demand for people who can decide what to build. AI Product Management has a bright future! Software is often written by teams that comprise Produc

In [5]:
images

[LoadedImage(url=https://dl-staging-website.ghost.io/content/images/2025/01/AIProductManager-2_1200px-1.jpg),
 LoadedImage(url=https://dl-staging-website.ghost.io/content/images/2025/01/The-Batch-ads-and-exclusive-banners---2024-12-16T174314.640--1-.png),
 LoadedImage(url=https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--45-.png),
 LoadedImage(url=https://dl-staging-website.ghost.io/content/images/2025/01/BIDENCHIPS-10_1200px.jpg),
 LoadedImage(url=https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--47-.jpg),
 LoadedImage(url=https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--44-.gif)]

In [6]:
text_splitter = RecursiveTextSplitter()
image_describer = BLIPImageDescriber()

text_docs = text_splitter.split(text, loaded_data.url)
image_docs = [image_describer.describe(image) for image in images]
docs = text_docs + image_docs
docs

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


[TextDocument(id='b0c2a28ce65fadc891e9f1f7388f8766c580740bbedcd365f2ad3429e2dc03c4-e12cdda1-d20d-4c44-91ae-b7c3ee4447df', content='issue 284\nNews\nDeepSeek Ups the Open Weights Ante\nU.S. Moves to Expand AI Export Restrictions\nAI Supercomputer on Your Desk\nCalibrating Contrast\n\n\nExplore Courses\nAI Newsletter\n\nCommunity\n\nResources\nCompany', source_url='https://www.deeplearning.ai/the-batch/issue-284/'),
 TextDocument(id='223c5a21f701afa70a873faff243339d9b591ef583a3e35e76ea897519c34a57-ecd6515e-8160-4e39-a4f1-78867c0363d8', content="✨ New course! Enroll in Reinforcement Fine-Tuning LLMs with GRPO Explore Courses AI Newsletter The Batch Andrew's Letter Data Points ML Research Blog Community Forum Events Ambassadors Ambassador Spotlight Resources Company About Careers Contact Start Learning Weekly Issues Andrew's Letters Data Points ML Research Business Science Culture Hardware AI Careers About Subscribe The Batch Weekly Issues issue 284 Published Jan 15, 2025 Reading time 13 m

In [7]:
embedding_func = SentenceTransformerTextEmbedding()
docs_embeddings = np.vstack([embedding_func.encode([doc.content]) for doc in docs])
docs_embeddings

[2025-05-24 23:41:55,443 | sentence_transformers.SentenceTransformer | INFO] -> Use pytorch device_name: cpu
[2025-05-24 23:41:55,444 | sentence_transformers.SentenceTransformer | INFO] -> Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Batches: 100%|██████████| 1/1 [00:00<00:00, 160.24it/s]


array([[-0.07752761, -0.07712671, -0.00187728, ..., -0.00018483,
        -0.04130648,  0.03334473],
       [-0.07596719, -0.04356242,  0.04169593, ..., -0.04087131,
        -0.01327076,  0.02455314],
       [-0.00362282, -0.09913214,  0.05776122, ..., -0.01191531,
         0.0412215 ,  0.01541616],
       ...,
       [ 0.09326685,  0.0327084 , -0.00491121, ..., -0.02058907,
        -0.0873061 , -0.03950423],
       [ 0.02394519,  0.0708113 , -0.04790877, ...,  0.0354294 ,
        -0.01772317, -0.02503958],
       [-0.06404288,  0.10308387, -0.06203021, ...,  0.03419654,
         0.03382072,  0.08427884]], shape=(101, 384), dtype=float32)

In [8]:
adapted_embedding_func = ChromaTextEmbeddingAdapter(embedding_function=SentenceTransformerTextEmbedding())
vectorstore = ChoromaVectorStore(collection_name='new_collection', 
                                 embedding_function=adapted_embedding_func)

[2025-05-24 23:42:00,970 | sentence_transformers.SentenceTransformer | INFO] -> Use pytorch device_name: cpu
[2025-05-24 23:42:00,971 | sentence_transformers.SentenceTransformer | INFO] -> Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
c:\Users\User\Desktop\MultimodalRAGSystem\VectorStore\chroma_vector_store.py:59: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectorstore = Chroma(embedding_function=self.embedding_function,
[2025-05-24 23:42:03,366 | chromadb.telemetry.product.posthog | INFO] -> Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [9]:
vectorstore.add_documents(documents=docs,
                          embeddings=docs_embeddings)

In [10]:
query = "What is computer?"
vectorstore.similarity_search(query, k=5)

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.84it/s]


[ImageDocument(id='848f8ad5d57320b6a79a9ff5ec1244061baa479fac35136662280df95e1cd4eb-b791ebba-5e40-4977-9d2a-5c79afd6eb04', content='a cartoon depicting two men in front of a computer', image=None, source_url='https://dl-staging-website.ghost.io/content/images/2025/01/AIProductManager-2_1200px-1.jpg', image_url='https://dl-staging-website.ghost.io/content/images/2025/01/AIProductManager-2_1200px-1.jpg'),
 TextDocument(id='3606b3a086420ec6180eff7a8613d6e277c849638e3fa4668e562f5799a5e5fd-a9439378-aa62-4496-b187-eac1d68284eb', content='can be connected to run models such as Meta’s Llama 3.1 405B. Complete specifications are not yet available. Project Digits runs Nvidia’s DGX operating system, a flavor of Ubuntu Linux. The system is based on a GB10 system-on-a-chip that combines the Nvidia Blackwell GPU architecture (which serves as the basis for its latest B100 GPUs) and Grace CPU architecture (designed to manage AI workloads in data centers), connected via high-bandwidth NVLink interconne

In [11]:
from langchain.prompts import PromptTemplate

News_article_prompt_template = PromptTemplate(
    input_variables=["context", "user_query"],
    template="""
You are an advanced AI assistant designed to provide comprehensive and well-reasoned answers to user queries based on relevant news articles. You have access to the following context, which may include text from news articles and associated images (described in text form).

Context:
{context}

Instructions:
- Carefully read the context above, which contains multiple news articles relevant to the user's question.
- If any images are described (e.g., captions, OCR, extracted text), incorporate them into your analysis.
- Summarize and synthesize the information to answer the user's question.
- Ensure your answer is accurate, relevant, and concise.

User query:
{user_query}

Your answer:
"""
)

In [12]:
rag_llm = OllamaRAGLLM(vectorstore=vectorstore, prompt_template=News_article_prompt_template)

C:\Users\User\AppData\Local\Temp\ipykernel_15124\194367445.py:1: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  rag_llm = OllamaRAGLLM(vectorstore=vectorstore, prompt_template=News_article_prompt_template)


In [13]:
user_query = 'computer'
res = rag_llm.query(user_query)

Batches: 100%|██████████| 1/1 [00:00<00:00, 75.77it/s]


In [14]:
res

RAGLLMResponse(user_query='computer')

In [15]:
print(res.llm_resopnse)

Based on the context provided, there is no direct mention of a "computer" in relation to the topic of AI models and export controls. However, it can be inferred that the context is discussing advanced computing systems, such as GPUs (Graphics Processing Units) and system-on-a-chip devices like the GB10.

The Project Digits system, for example, is described as a high-performance computing platform designed for running large-scale AI models. It consists of a GB10 system-on-a-chip that combines Nvidia's Blackwell GPU architecture with Grace CPU architecture, connected via high-bandwidth NVLink interconnect. The system comes with 128 GB of unified memory and 4 terabytes of solid-state storage.

While the context does not directly mention computers, it is clear that advanced computing systems are being developed to support the growing demand for AI processing power. Nvidia's DGX operating system, which runs on the Project Digits platform, is mentioned as a flavor of Ubuntu Linux, suggesting

In [16]:
res.relevant_docs

[ImageDocument(id='848f8ad5d57320b6a79a9ff5ec1244061baa479fac35136662280df95e1cd4eb-b791ebba-5e40-4977-9d2a-5c79afd6eb04', content='a cartoon depicting two men in front of a computer', image=None, source_url='https://dl-staging-website.ghost.io/content/images/2025/01/AIProductManager-2_1200px-1.jpg', image_url='https://dl-staging-website.ghost.io/content/images/2025/01/AIProductManager-2_1200px-1.jpg'),
 TextDocument(id='b53fd95126f3947324d73cfd39e8688ee7f59790ee57a2e7ef85eeb2a4f45c82-933e940e-8121-477f-b4cc-a7dc3e9e0107', content='The U.S. Commerce Department’s export control agency must approve the export of models or transfer of weights of closed models that were trained using more than 10 26 computational operations. These rules target future systems, as no known models today used this amount of computation during training.\nCompanies based in the U.S. must maintain at least 50 percent of their total AI computing power within U.S. borders. They also must track distribution of their